In [2]:
from fhir.resources.bundle import Bundle 
import json
import jmespath
from kafka import KafkaProducer
import glob

In [3]:
!docker compose stop
!docker compose rm -f
!docker compose up -d

WARN[0000] /Users/soufiane/Workspace/agile/flink_sql_tutorial/docker-compose.yml: `version` is obsolete 
WARN[0000] /Users/soufiane/Workspace/agile/flink_sql_tutorial/docker-compose.yml: `version` is obsolete 
No stopped containers
WARN[0000] /Users/soufiane/Workspace/agile/flink_sql_tutorial/docker-compose.yml: `version` is obsolete 
[+] Running 0/0
 ⠋ kafka-ui Pulling                                                        0.1s 
 ⠋ adminer Pulling                                                         0.1s 
 ⠋ kafka-edge1 Pulling                                                     0.1s 
 ⠋ zookeeper-edge1 Pulling                                                 0.1s 
 ⠋ db Pulling                                                              0.1s 
[+] Running 0/5
 ⠙ kafka-ui Pulling                                                        0.2s 
 ⠙ adminer Pulling                                                         0.2s 
 ⠙ kafka-edge1 Pulling                                          

In [6]:
def load_fhir_resources(f):
    with open(f, "r") as file:
        data = json.load(file)
    resources = jmespath.search("entry[*]", data)
    return resources
def send_resources_to_kafka(resources):
    producer = KafkaProducer(bootstrap_servers='localhost:9095', value_serializer=lambda v: json.dumps(v).encode('utf-8'))
    for resource in resources:
        if "fullUrl" in resource:
            print("{} - {} ".format(resource["resource"]["resourceType"], resource["fullUrl"]))
        else:
            print(resource["id"])
        producer.send(resource["resource"]["resourceType"], key=resource["fullUrl"].encode(), value=resource["resource"])
    print("waiting for flush")
    producer.flush()    

## Patient

In [7]:
for f in glob.glob("output/fhir/*"):
    if ("hospitalInformation" not in f) and ("practitionerInformation" not in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)

Loading output/fhir/Guadalupe206_Quigley282_07f63393-cea1-14ed-2407-d241fd884f30.json
Patient - urn:uuid:07f63393-cea1-14ed-2407-d241fd884f30 
Encounter - urn:uuid:9345fe29-0462-726d-de91-4826146ce89f 
Condition - urn:uuid:a3ffcafb-2ece-5ccb-a859-6282ec0abd27 
DiagnosticReport - urn:uuid:34325ab0-2ace-39da-8dbc-04b480f6c484 
DocumentReference - urn:uuid:46aaac84-f13f-e6cb-8aff-ff513e8f097c 
Claim - urn:uuid:831b7c8e-7716-1423-e92e-e9cef55020b5 
ExplanationOfBenefit - urn:uuid:a4f29c45-015b-7736-fd7d-615b1fdb4e2b 
Encounter - urn:uuid:c4ba790f-7bbd-aa3d-b797-de11d5d0482c 
Condition - urn:uuid:efb2cfab-1ed5-f8cf-8f92-b1164cfe391a 
Procedure - urn:uuid:7e7e30d4-ea7e-4a46-fe1d-51c1c6bb3efe 
Procedure - urn:uuid:1e9b31ea-0175-1aa5-988e-99587ec75c87 
Procedure - urn:uuid:96cfe9ff-de24-944c-fe16-c4bd841dffe4 
Procedure - urn:uuid:9ed91db6-2d9b-7587-6637-d2ca1309144e 
Procedure - urn:uuid:6bafb2e1-ae9a-cc0e-268f-1a1e5033f436 
Procedure - urn:uuid:6a68f26b-b234-f969-62c1-fd39b16b6d4d 
Procedure

## Practioner

In [8]:
for f in glob.glob("output/fhir/*"):
    if ("practitionerInformation"  in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)

Loading output/fhir/practitionerInformation1719204487096.json
Practitioner - urn:uuid:69e2ceda-7632-3d0f-90a1-b9dbf9aa0551 
PractitionerRole - urn:uuid:fa230203-bf12-d13b-b811-cc0c6706b57d 
Practitioner - urn:uuid:8356ef49-ffce-3952-a000-4c1bc3115be2 
PractitionerRole - urn:uuid:0c461980-a6a7-df54-c437-c4d23dc63dc2 
Practitioner - urn:uuid:c066c852-cbfd-3024-ae81-3f919ef8fa65 
PractitionerRole - urn:uuid:b2179df0-1973-6425-c3fd-26b1467799e4 
waiting for flush


## Hospital

In [9]:
for f in glob.glob("output/fhir/*"):
    if ("hospitalInformation"  in f): 
        print(f"Loading {f}")
        resources = load_fhir_resources(f)
        send_resources_to_kafka(resources)

Loading output/fhir/hospitalInformation1719204487096.json
Organization - urn:uuid:db28cc9a-fdfb-35a6-aef7-ab9b933ef244 
Location - urn:uuid:9a277a7c-edef-39ed-a619-58dc818c416d 
Organization - urn:uuid:3d5fbf38-c781-3e04-8c7e-5a402611b7d4 
Location - urn:uuid:044ef112-a176-3686-8385-f2af139c51bd 
Organization - urn:uuid:bac9dc0c-cb0f-3fa6-aa12-819dd0fb922a 
Location - urn:uuid:580dfad8-45bd-3de0-8636-3140d44a4ba5 
Location - urn:uuid:bb1ad573-19b8-9cd8-68fb-0e6f684df992 
waiting for flush
